# Art Institute of Chicago web scraper

In [ ]:
# Terms of use

# include AIC-User-Agent header:
headers = {"AIC-User-Agent": "artworks-semantic-search-application (annabozhenko2003@gmail.com)"}

In [1]:
from pathlib import Path 
import os

import json

from datasets import load_dataset

d:\artworks_search_engine\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Explore artpiece json structure

In [2]:
chicago_collection_dir = Path(r"D:\artworks_search_engine\data_retrieval\artic-api-data\json\artworks")


In [ ]:
# which entities are in the chicago  aside 
# from Prints and drawings and paintings
# 
# 

In [ ]:
b

In [5]:
important_fields = [
'title',
'place_of_origin',
'medium_display',
'classification_titles',
'subject_titles',
'style_title', # handle null
'technique_titles',
'term_titles',
'material_titles',
'description',
'category_titles',
'artist_title',
'id',
'date_start',
'date_end',
'department_title',
'image_id']


In [6]:
chicago_collection_dir = Path(r"D:\artworks_search_engine\data_retrieval\artic-api-data\json\artworks")
artworks = {field: [] for field in important_fields}

for f in os.listdir(chicago_collection_dir):
    with open(Path(chicago_collection_dir, f)) as f:
        artwork = json.load(f)
        [artworks[field].append(artwork[field]) for field in important_fields]
        

In [7]:
from datasets import Dataset

In [8]:
ds = Dataset.from_dict(artworks)

In [9]:
ds

Dataset({
    features: ['title', 'place_of_origin', 'medium_display', 'classification_titles', 'subject_titles', 'style_title', 'technique_titles', 'term_titles', 'material_titles', 'description', 'category_titles', 'artist_title', 'id', 'date_start', 'date_end', 'department_title', 'image_id'],
    num_rows: 134078
})

In [72]:
# list_ts = [
#  'classification_titles',
#  'subject_titles',
#  'technique_titles',
#  'material_titles',
#  'category_titles',
#  'term_titles']

# ds = ds.map(lambda x: {f: ", ".join(x[f]) for f in list_ts})


Map: 100%|██████████| 134078/134078 [00:30<00:00, 4329.00 examples/s]


In [10]:
ds = ds.rename_columns({'classification_titles': 'classification',
                   'subject_titles': 'subjects',
                   'technique_titles': 'techniques',
                   'material_titles': 'materials',
                   'category_titles': 'categories',
                   'term_titles': 'tags',
                   'artist_title': 'artist',
                   })

In [11]:
N = len(ds)

ds = ds.add_column("museum", ["The Art Institute of Chicago"] * N)
ds = ds.add_column("museum_region", ["Chicago, Illinois"] * N)
ds = ds.add_column("museum_country", ["USA"] * N)


In [12]:
ds = ds.map(lambda batch: {"image_url": [f"https://www.artic.edu/iiif/2/{x}/full/843,/0/default.jpg" for x in batch['image_id']]},
            batched=True,
            batch_size=1000)


Map: 100%|██████████| 134078/134078 [00:36<00:00, 3656.81 examples/s]


In [1]:
from datasets import load_from_disk


d:\artworks_search_engine\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chicago_ds = load_from_disk("chicago_ds")


In [6]:
N = len(chicago_ds)

chicago_ds = chicago_ds.add_column("museum", ["The Art Institute of Chicago"] * N)
chicago_ds = chicago_ds.add_column("museum_region", ["Chicago"] * N)
chicago_ds = chicago_ds.add_column("museum_country", ["USA"] * N)


In [15]:
ds[0]

{'title': 'Simon Vouet',
 'place_of_origin': 'Flanders',
 'medium_display': 'Engraving in black on cream laid paper',
 'classification': ['engraving', 'print', 'prints and drawing'],
 'subjects': [],
 'style_title': None,
 'techniques': [],
 'tags': ['engraving', 'paper (fiber product)', 'print', 'prints and drawing'],
 'materials': ['paper (fiber product)'],
 'description': None,
 'categories': ['Prints and Drawings'],
 'artist': 'Robert van Voerst',
 'id': 100,
 'date_start': 1630,
 'date_end': 1636,
 'department_title': 'Prints and Drawings',
 'image_id': '03c0fd45-3690-5fae-8023-4a3c16d3bfad',
 'museum': 'The Art Institute of Chicago',
 'museum_region': 'Chicago, Illinois',
 'museum_country': 'USA',
 'image_url': 'https://www.artic.edu/iiif/2/03c0fd45-3690-5fae-8023-4a3c16d3bfad/full/843,/0/default.jpg'}

In [3]:
full_info_fields = [
 'title',
 'classification',
 'subjects',
 'techniques',
 'materials',
 'description',
 'categories',
 'artist',
 'date_start',
 'date_end'
]

In [16]:
ds = ds.map(lambda batch: {"url": [f"https://www.artic.edu/artworks/{x}" for x in batch["id"]]},
                            batched = True,
                            batch_size=1000)

Map: 100%|██████████| 134078/134078 [00:00<00:00, 234776.89 examples/s]


In [17]:
ds.save_to_disk("./chicago_ds")


Saving the dataset (1/1 shards): 100%|██████████| 134078/134078 [00:00<00:00, 304137.65 examples/s]


# Redundant code (very specific) start

In [7]:
 
def safe_string(x):
    return '' if x is None else x

chicago_ds = chicago_ds.map(lambda x: {"full_info": ". ".join([f"{f}: {safe_string(x[f])}"
                                                for f in full_info_fields]) + f". museum: The Art Institute of Chicago, location: Chicago, USA"})


Map: 100%|██████████| 134078/134078 [00:40<00:00, 3283.28 examples/s]


In [ ]:
 
def safe_string(x):
    return '' if x is None else x

ds = ds.map(lambda x: {"full_info": ". ".join([f"{f}: {safe_string(x[f])}" 
                                                for f in full_info_fields]) + f". museum: The Art Institute of Chicago, location: Chicago, USA"})


In [ ]:
ds = ds.map(lambda batch: {"url": [f"https://www.artic.edu/artworks/{x}" for x in batch["id"]]},
                            batched = True,
                            batch_size=1000,
                            num_proc=num_proc)


In [11]:
ds.save_to_disk("./chicago_ds0")


Saving the dataset (1/1 shards): 100%|██████████| 134078/134078 [00:00<00:00, 402859.09 examples/s]


In [10]:
ds.save_to_disk("./chicago_ds")


NameError: name 'ds' is not defined

# Redundant code (very specific) end